#LỌC CỘNG TÁC DỰA TRÊN USER

Là thuật toán dựa trên số điểm mà một nhóm user tương đồng nhất với user mục tiêu đã đánh giá cho item $j$, từ đó ta dự đoán được số điểm mà user mục tiêu có thể đánh giá cho item $j$ để đưa ra quyết định có nên giới thiệu item $j$ cho user mục tiêu hay không.

### **Ví dụ**
(Bài tập 2 trang 69 sách Recommender System của Springer)

Ta có một ma trận rating gồm 5 users (theo hàng) và 6 (theo cột) items như sau:

ItemId->|1|2|3|4|5|6|
-|:-:|:-:|:-:|:-:|:-:|:-:|
1|5|6|7|4|3|Nan|
2|4|Nan|3|Nan|5|4|
3|Nan|3|4|1|1|Nan|
4|7|4|3|6|Nan|4|
5|1|Nan|3|2|2|5|

*Yêu cầu*: dự đoán giá trị bị khuyết của user 2 bằng thuật toán lọc cộng tác dựa trên user.  Sử dụng hệ số tương quan Pearson và mean-centering (tạo biến mới bằng chính giá trị biến đó trừ đi giá trị trung bình).
Chọn $k=2$.






#### **Các bước thực hiện:**
1. Chuẩn hóa dữ liệu:
  - Tính giá trị trung bình của mỗi user dựa trên rating quan sát được
$$\mu_u = \frac{\sum_{k \in I_u} r_{uk}}{|I_u|}, \forall u \in \{1,...,m\}.$$
trong đó $I_u$ là tập hợp gồm các chỉ số item mà được đánh giá bởi user $u$.
  - Trừ số điểm rating của từng item mà user $u$ đã đánh giá cho giá trị trung bình của user $u$.
  $$s_{uj} = r_{uj} - \mu_u, \forall u \in \{1,...,m\}.$$
2. Tính hệ số tương quan Pearson
$$\text{Sim}(u,v) = \text{Pearson}(u,v) = \frac{\sum_{k \in I_u \cap I_v} (r_{uk} - \mu_u)\cdot(r_{vk} - \mu_v)}{\sqrt{\sum_{k \in I_u \cap I_v} (r_{uk} - \mu_u)^2} \cdot \sqrt{\sum_{k \in I_u \cap I_v} (r_{vk} - \mu_v)^2}}.$$
3. Chọn k users tương đồng với user mục tiêu
4. Dự đoán rating của user mục tiêu đối với các item mà user mục tiêu chưa đánh giá.  
$$\hat r_{uj} = \mu_u + \frac{\sum_{v \in P_u(j)} \text{Sim}(u,v) \cdot (r_{vj} - \mu_v)}{\sum_{v \in P_u(j)} |\text{Sim}(u,v)|}.$$
với $P_u(j)$ là tập $k$ users tương đồng nhất với user mục tiêu $u$ mà đã đánh giá item $j$.

**Trường hợp không mean-centering**
$$\hat r_{uj} = \frac{\sum_{v \in P_u(j)} \text{Sim}(u,v) \cdot r_{vj}}{\sum_{v \in P_u(j)} |\text{Sim}(u,v)|}.$$

Cụ thể, trong bài toán này, user mục tiêu là user 2. Cần dự đoán số điểm user 2 sẽ đánh giá item 2 và 4 dựa trên $k=2$ user tương đồng nhất với user 2.

ItemId->|1|2|3|4|5|6|Mean Rating|Pearson(i,2)|
-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
1|0|1|2|-1|-2|Nan|5|-1|
2|0|Nan|-1|Nan|1|0|4|1|
3|Nan|0.75|1.75|-1.25|-1.25|Nan|2.25|-0.986|
4|2.2|-0.8|-1.8|1.2|Nan|-0.8|4.8|0.610|
5|-1.6|Nan|0.4|-0.6|-0.6|2.4|2.6|-0.238|

$\text{Pearson}(3,2) = \frac{(1.75)*(-1)+(-1.25)*(1)}{\sqrt{(1.75)^2 + (-1.25)^2}*\sqrt{(-1)^2+(1)^2}} = \frac{-3}{\sqrt{4.625}*{\sqrt{2}}} \approx -0.986$

$\text{Pearson}(4,2) = \frac{(2.2)*(0)+(-1.8)*(-1)+(-0.8)*(0)}{\sqrt{(2.2)^2+(-1.8)^2+(-0.8)^2}*\sqrt{(0)^2+(-1)^2+(0)^2}} \approx 0.6095$

$\hat r_{2,2} = \mu_2+ \frac{Sim(4,2) \cdot (r_{4,2} - \mu_4)}{Sim(4,2)} = 4 + \frac{0.610*(-0.8)}{0.610} = 3.2$

$\hat r_{2,4} = \mu_2 + \frac{Sim(4,2) \cdot (r_{4,4} - \mu_4)}{Sim(4,2)} = 4 + \frac{0.610*(1.2)}{0.610} = 5.2$

####**Sử dụng Python**

Có thể chia nhỏ bài toán thành các bước sau:

- Bước 1: Tính ma trận hệ số tương quan
- Bước 2: Chọn user mục tiêu và loại bỏ user mục tiêu ra khỏi ma trận hệ số tương quan
- Bước 3: Chọn k users tương đồng với user mục tiêu
- Bước 4: Lọc ra các item mà nhóm user tương đồng đã đánh giá, đồng thời loại bỏ items mà không user nào đánh giá
- Bước 5: Loại bỏ item mà user mục tiêu đã đánh giá trong tập item ở bước 4
- Bước 6: Tính rating của từng item và chọn ra n item có số điểm cao nhất

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [16]:
# Ma trận rating
mx = np.array([[5,6,7,4,3,np.nan],
              [4,np.nan,3,np.nan,5,4],
              [np.nan,3,4,1,1,np.nan],
              [7,4,3,6,np.nan,4],
              [1,np.nan,3,2,2,5]])

# Chuyển ma trận mx thành dataframe
example_df = pd.DataFrame(mx, index=['user1','user2','user3','user4','user5'], columns=['item1','item2','item3','item4','item5','item6'])
example_df

,item1,item2,item3,item4,item5,item6
user1,5.0,6.0,7.0,4.0,3.0,NaN
user2,4.0,NaN,3.0,NaN,5.0,4.0
user3,NaN,3.0,4.0,1.0,1.0,NaN
user4,7.0,4.0,3.0,6.0,NaN,4.0
user5,1.0,NaN,3.0,2.0,2.0,5.0


In [17]:
def pearson_corr(user_u, user_v):
  """
  Hàm tính hệ số tương quan Pearson
  @param: user_u, user_v: 2 vector cần tính hệ số tương quan Pearson
  @return: mức độ tương đồng giữa 2 vector
  """

  # Tìm trung bình rating của user u và v
  u_avg = np.nansum(user_u)/np.count_nonzero(~np.isnan(user_u))
  v_avg = np.nansum(user_v)/np.count_nonzero(~np.isnan(user_v))
  # Tìm index mà user_u và user_v đều có rating
  mutual_index = [i for i in range(len(user_u)) if not np.isnan(user_u[i]) and not np.isnan(user_v[i])]

  # Tạo biến để lưu giá trị của tử số và mẫu số
  dividend = 0
  u_divisor = 0
  v_divisor = 0

  # Tính hệ số tương quan Pearson
  for index in mutual_index:
    u_nr = user_u[index] - u_avg # mean-centering rating của user_u
    v_nr = user_v[index] - v_avg # mean-centering rating của user_v
    dividend += u_nr * v_nr      # Tính tử số của Pearson
    u_divisor += pow(u_nr, 2)
    v_divisor += pow(v_nr, 2)

  divisor = np.sqrt(u_divisor) * np.sqrt(v_divisor) # Mẫu số của Pearson

  if divisor != 0:
    return round(dividend / divisor, 3) # hệ số tương quan Pearson nếu mẫu số != 0

  return 0 # trả về 0 nếu mẫu số = 0

def corr_matrix(data):
  """
  Hàm trả về ma trận hệ số tương quan
  @param:
    data: dataframe với dòng ~ users, cột ~ items
  @return: dataframe hệ số tương quan giữa các user có trong data
  """
  m, n = data.shape
  corr_mx = np.identity(m) # ma trận đơn vị có số chiều bằng số lượng user
  for i in range(m-1):
    for j in range(i+1,m):
      corr_mx[i,j] = corr_mx[j,i] = pearson_corr(data.iloc[i].tolist(), data.iloc[j].tolist())
  corr_mx = pd.DataFrame(corr_mx, index=data.index, columns=data.index)

  return corr_mx

In [18]:
corr_matrix(example_df)

,user1,user2,user3,user4,user5
user1,1.000,-1.000,0.974,-0.717,0.467
user2,-1.000,1.000,-0.986,0.610,-0.238
user3,0.974,-0.986,1.000,-0.999,0.943
user4,-0.717,0.610,-0.999,1.000,-0.726
user5,0.467,-0.238,0.943,-0.726,1.000


In [27]:
# Bước 1 đến bước 5
def filtering(data, target_user, k, threshold=0.2):
  """
  Hàm lọc k user tương đồng với user mục tiêu và tập các item mà user mục tiêu chưa đánh giá
  @params:
    data: dataframe với dòng ~ users, cột ~ items
    target_user: user mục tiêu
    k: số lượng user tương đồng với user mục tiêu
    threshold: ngưỡng hệ số tương đồng thấp nhất
  @return:
    similar_users: tập các users tương đồng nhất với user mục tiêu
    similar_user_rated: tập item nhóm user tương đồng đã đánh giá nhưng user mục tiêu chưa đánh giá
  """

  # Bước 1: ma trận hệ số tương quan
  pearson_mx = corr_matrix(data)

  # Bước 2: Loại bỏ user mục tiêu ra khỏi ma trận hệ số tương quan
  pearson_mx = pearson_mx.drop(index=target_user)

  # Bước 3: Chọn k users tương đồng với user mục tiêu
  similar_users = (pearson_mx[pearson_mx[target_user]>threshold][target_user]
                   .sort_values(ascending=False)[:k])

  # Bước 4: Các item mà nhóm user tương đồng đã đánh giá, đồng thời loại bỏ items mà không user nào đánh giá
  similar_user_rated = data[data.index.isin(similar_users.index)].dropna(axis=1, how='all')

  # Bước 5: Loại bỏ item mà user mục tiêu đã đánh giá trong tập item tìm được bước 4
  target_user_rated = data[data.index == target_user].dropna(axis=1, how='all') ### các Item mà target_user đã đánh giá
  similar_user_rated_df = similar_user_rated.drop(target_user_rated.columns, axis=1, errors='ignore')

  return (similar_users, similar_user_rated_df)

In [32]:
# Bước 6: dự đoán số điểm mà user mục tiêu có thể đánh giá
def pred_score(data, target_user, similar_users, similar_user_rated, mean_centered=True):
  """
  Hàm dự đoán số điểm mà user mục tiêu có thể đánh giá
  @params:
    data: dataframe với dòng ~ users, cột ~ items
    target_user: user mục tiêu
    similar_users: similar_users: tập các users tương đồng nhất với user mục tiêu
    similar_user_rated: tập item nhóm user tương đồng đã đánh giá nhưng user mục tiêu chưa đánh giá
  @return:
    pred_score: dataframe gồm các item và số điểm rating dự đoán tương ứng
  """

  # Rating trung bình của target_user dựa trên dữ liệu quan sát được
  avg_rating = data[data.index == target_user].T.mean()[target_user] if mean_centered == True else 0
  # Khởi tạo dictionary để lưu điểm của item
  pred_score = {}

  # Lặp qua từng item mà nhóm user tương đồng đã đánh giá
  for i in similar_user_rated.columns:
    # Rating của item i
    item_rating = similar_user_rated[i]

    # Tạo biến để lưu giá trị của tử số và mẫu số
    nominator = 0
    denominator = 0
    # Lặp qua các user tương đồng
    for u in similar_users.index:
      # Chỉ tính khi rating != nan
      if pd.isna(item_rating[u]) == False:
        user_avg = data[data.index == u].T.mean()[u] if mean_centered == True else 0
        score = similar_users[u] * (item_rating[u]-user_avg)
        nominator += score
        denominator += similar_users[u]

    # Dự đoán rating
    pred_score[i] = avg_rating + (nominator / denominator)

  # Chuyển dictionary thành dataframe và sắp xếp dữ liệu theo prediction_score
  pred_score_df = pd.DataFrame(pred_score.items(), columns=['item', 'pred_score']).sort_values(by='pred_score', ascending=False)

  return pred_score_df

In [21]:
k = 2
target_user = 'user2'
similar_users, similar_user_rated = filtering(example_df, target_user, k, 0.2)
display(similar_users)
display(similar_user_rated)
pred_score = pred_score(example_df, target_user, similar_users, similar_user_rated, mean_centered=True)
pred_score

user4    0.61
Name: user2, dtype: float64

,item2,item4
user4,4.0,6.0


,item,pred_score
1,item4,5.2
0,item2,3.2


### **Bài tập**

Tập dữ liệu gồm 25 users (25 dòng) và 100 movies (100 cột).

Link file: https://docs.google.com/spreadsheets/d/1puYa6eqTf21PA2kR0mHif6oXViprqCSA/edit?usp=sharing&ouid=115422524039268283690&rtpof=true&sd=true

Trả lời các câu hỏi sau:
1. Tính ma trận tương quan bằng hệ số tương quan Pearson.
2. Xác định 5 users tương đồng nhất với user 3867 và user 89.
3. Dự đoán số điểm đánh giá của user 3867 và user 89 cho từng bộ phim (chưa được đánh giá bởi user tương ứng).

Thực hiện 3 câu trên trong 2 trường hợp sau:
- TH1: Không chuẩn hóa dữ liệu
- TH2: Chuẩn hóa dữ liệu bằng mean-centering (trừ số điểm rating quan sát được cho giá trị rating trung bình của user tương ứng)




In [9]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

# Thay đổi đương dẫn
import os
os.chdir("drive/My Drive/KHTN/RecSys/data")

# Print out the current directory
!pwd

Mounted at /content/drive
/content/drive/My Drive/KHTN/RecSys/data


In [29]:
# Load the dataset
data = pd.read_excel('data_01.xls')
data.rename(columns={"Unnamed: 0": "userId"}, inplace=True)
data.set_index('userId', inplace = True)
data.head()

,11: Star Wars: Episode IV - A New Hope (1977),12: Finding Nemo (2003),13: Forrest Gump (1994),14: American Beauty (1999),22: Pirates of the Caribbean: The Curse of the Black Pearl (2003),24: Kill Bill: Vol. 1 (2003),38: Eternal Sunshine of the Spotless Mind (2004),63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995),77: Memento (2000),85: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),...,8467: Dumb & Dumber (1994),8587: The Lion King (1994),9331: Clear and Present Danger (1994),9741: Unbreakable (2000),9802: The Rock (1996),9806: The Incredibles (2004),10020: Beauty and the Beast (1991),36657: X-Men (2000),36658: X2: X-Men United (2003),36955: True Lies (1994)
userId,,,,,,,,,,,,,,,,,,,,,
1648,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,5.0,3.5,3.0,NaN,3.5,NaN
5136,4.5,5.0,5.0,4.0,5.0,5.0,5.0,3.0,NaN,5.0,...,1.0,5.0,NaN,NaN,NaN,5.0,5.0,4.5,4.0,NaN
918,5.0,5.0,4.5,NaN,3.0,NaN,5.0,NaN,5.0,NaN,...,NaN,5.0,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN
2824,4.5,NaN,5.0,NaN,4.5,4.0,NaN,NaN,5.0,NaN,...,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3867,4.0,4.0,4.5,NaN,4.0,3.0,NaN,NaN,NaN,4.5,...,1.0,4.0,NaN,NaN,NaN,3.0,4.0,4.0,3.5,3.0


In [38]:
k = 5
target_user = 89
similar_users, similar_user_rated = filtering(data, target_user, k, 0.2)
display(similar_users)

######### Có chuẩn hoá ##########
mean_centered_pred_score = pred_score(data, target_user, similar_users, similar_user_rated, mean_centered=True)
#display(mean_centered_pred_score.head(10))

######### Không chuẩn hoá ##########
without_mean_centered_pred_score = pred_score(data, target_user, similar_users, similar_user_rated, mean_centered=False)
#display(without_mean_centered_pred_score.head(10))

pd.merge(mean_centered_pred_score, without_mean_centered_pred_score, how='inner', on='item', suffixes=('_mean_centered', '_without_mean_centered')).round(3).head(10)

userId
4809    0.655
860     0.541
5062    0.512
5136    0.507
3525    0.437
Name: 89, dtype: float64

,item,pred_score_mean_centered,pred_score_without_mean_centered
0,424: Schindler's List (1993),5.204,4.731
1,807: Seven (a.k.a. Se7en) (1995),5.201,4.771
2,122: The Lord of the Rings: The Return of the ...,5.193,4.700
3,120: The Lord of the Rings: The Fellowship of ...,5.017,4.524
4,121: The Lord of the Rings: The Two Towers (2002),5.017,4.524
5,77: Memento (2000),4.977,4.328
6,568: Apollo 13 (1995),4.928,4.371
7,8587: The Lion King (1994),4.921,4.492
8,274: The Silence of the Lambs (1991),4.895,4.465
9,85: Raiders of the Lost Ark (Indiana Jones and...,4.841,4.348
